In [ ]:
import os
import time
import copy
import argparse
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug
import warnings
warnings.filterwarnings("ignore")

# watch -n 1 nvidia-smi
import os

# 显示第 0 和第 1 个 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
parser = argparse.ArgumentParser(description='Parameter Processing')
parser.add_argument('--dataset', type=str, default='CIFAR10', help='dataset')
parser.add_argument('--model', type=str, default='ConvNet', help='model')
parser.add_argument('--ipc', type=int, default=50, help='image(s) per class')
parser.add_argument('--eval_mode', type=str, default='SS', help='eval_mode') # S: the same to training model, M: multi architectures,  W: net width, D: net depth, A: activation function, P: pooling layer, N: normalization layer,



parser.add_argument('--num_exp', type=int, default=3, help='the number of experiments')



parser.add_argument('--num_eval', type=int, default=1, help='the number of evaluating randomly initialized models')
parser.add_argument('--epoch_eval_train', type=int, default=1000, help='epochs to train a model with synthetic data') # it can be small for speeding up with little performance drop
parser.add_argument('--Iteration', type=int, default=2000, help='training iterations')
parser.add_argument('--lr_img', type=float, default=1.0, help='learning rate for updating synthetic images')
parser.add_argument('--lr_net', type=float, default=0.01, help='learning rate for updating network parameters')
parser.add_argument('--batch_real', type=int, default=256, help='batch size for real data')
parser.add_argument('--batch_train', type=int, default=256, help='batch size for training networks')
parser.add_argument('--init', type=str, default='real', help='noise/real: initialize synthetic images from random noise or randomly sampled real images.')
parser.add_argument('--dsa_strategy', type=str, default='color_crop_cutout_flip_scale_rotate', help='differentiable Siamese augmentation strategy')
parser.add_argument('--data_path', type=str, default='/home/ssd7T/ZTL_gcond/data_cv', help='dataset path')
parser.add_argument('--save_path', type=str, default='/home/ssd7T/ztl_dm/gen', help='path to save results')
parser.add_argument('--dis_metric', type=str, default='ours', help='distance metric')

args = parser.parse_args([])
args.method = 'DM'
args.outer_loop, args.inner_loop = get_loops(args.ipc)
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.dsa_param = ParamDiffAug()
args.dsa = False if args.dsa_strategy in ['none', 'None'] else True

if not os.path.exists(args.data_path):
    os.mkdir(args.data_path)

if not os.path.exists(args.save_path):
    os.mkdir(args.save_path)

eval_it_pool = np.arange(0, args.Iteration+1, 2000).tolist() if args.eval_mode == 'S' or args.eval_mode == 'SS' else [args.Iteration] # The list of iterations when we evaluate models and record results.
print('eval_it_pool: ', eval_it_pool)
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)
model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)


accs_all_exps = dict() # record performances of all experiments
for key in model_eval_pool:
    accs_all_exps[key] = []

data_save = []
pairs_real = []
indexs_real = []

for exp in range(args.num_exp):
    # pairs_real = []
    # indexs_real = []
    exp = exp + 80
    print('\n================== Exp %d ==================\n '%exp)
    print('Hyper-parameters: \n', args.__dict__)
    print('Evaluation model pool: ', model_eval_pool)

    ''' organize the real dataset '''
    images_all = []
    labels_all = []
    indices_class = [[] for c in range(num_classes)]

    images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
    labels_all = [dst_train[i][1] for i in range(len(dst_train))]
    for i, lab in enumerate(labels_all):
        indices_class[lab].append(i)
    images_all = torch.cat(images_all, dim=0).to(args.device)
    labels_all = torch.tensor(labels_all, dtype=torch.long, device=args.device)



    for c in range(num_classes):
        print('class c = %d: %d real images'%(c, len(indices_class[c])))

    def get_images(c, n): # get random n images from class c
        idx_shuffle = np.random.permutation(indices_class[c])[:n]
        return images_all[idx_shuffle]
    
    def get_images_init(c, n,exp): # get random n images from class c
        # start_idx = i  # 指定起始索引 i
        # end_idx = i + n  # 计算结束索引（不包括结束索引）

        # 从指定的起始索引到结束索引获取元素
        idx_shuffle  = indices_class[c][exp:exp + n]

        # idx_shuffle = np.random.permutation(indices_class[c])[:n]
        return images_all[idx_shuffle],idx_shuffle

    for ch in range(channel):
        print('real images channel %d, mean = %.4f, std = %.4f'%(ch, torch.mean(images_all[:, ch]), torch.std(images_all[:, ch])))


    ''' initialize the synthetic data '''
    image_syn = torch.randn(size=(num_classes*args.ipc, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True, device=args.device)
    label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]

    if args.init == 'real':
        print('initialize synthetic data from random real images')
        for c in range(num_classes):
            reals,index = get_images_init(c, args.ipc,exp)
            reals = reals.detach().data
            pairs_real.append(reals)
            indexs_real.append(index)
            image_syn.data[c*args.ipc:(c+1)*args.ipc] = reals
            # pairs_real
    else:
        print('initialize synthetic data from random noise')

In [ ]:
img_real_test= torch.cat(pairs_real, dim=0)

In [ ]:
images_all.shape

In [ ]:
label_real_test = []
for i in range(int(len(indexs_real)/10)):
    # print(i)
    label_real_test_ = []
    for c in range(num_classes):
        idx_shuffle = indexs_real[c + i*10]
        label_real_test_.append(labels_all[idx_shuffle].to("cpu"))
        # print()
    # img_real = torch.from_numpy(np.concatenate(img_real, axis=0))
    label_real_test_ = torch.from_numpy(np.concatenate(label_real_test_, axis=0))
    label_real_test.append(label_real_test_)
label_real_test = torch.cat(label_real_test, dim=0)

In [ ]:
label_real_test.shape

In [ ]:
device = args.device

In [ ]:
img_syn = []
label_syn = []
img_real_train = []
label_real_train = []
# /home/ssd7T/ztl_dm/indexs_real_20.pt
for i in range(80):
    try:
    
        img_syn_ = torch.load(f'/home/ssd7T/ztl_dm/img_syn_{i}.pt')
        label_syn_ = torch.load(f'/home/ssd7T/ztl_dm/label_syn_{i}.pt')
        pairs_real_=torch.load(f'/home/ssd7T/ztl_dm/pairs_real_{i}.pt')
        indexs_real_=torch.load(f'/home/ssd7T/ztl_dm/indexs_real_{i}.pt')
        
        img_real_train_ = torch.cat(pairs_real_, dim=0)
        
    
        label_real_ = []
        for c in range(num_classes):
            idx_shuffle = indexs_real_[c]
            label_real_.append(labels_all[idx_shuffle].to("cpu"))
        # img_real = torch.from_numpy(np.concatenate(img_real, axis=0))
        label_real_train_ = torch.from_numpy(np.concatenate(label_real_, axis=0))
        # label_real_train_ = torch.cat(label_real_train_, dim=0)
        
        img_syn.append(img_syn_)
        label_syn.append(label_syn_)
        img_real_train.append(img_real_train_)
        label_real_train.append( label_real_train_)
        # if i == 3 or i == 22 or i == 42 or i == 62:
        #     pairs_real_=torch.load(f'pairs_real_{i}.pt')
        #     img_real_train_ = torch.cat(pairs_real_, dim=0)
        #     img_real_train.append(img_real_train_)
            
    except:
        pass


img_syn = torch.cat(img_syn, dim=0).to(device)
label_syn = torch.cat(label_syn, dim=0).to(device)
img_real_train = torch.cat(img_real_train, dim=0).to(device)
label_real_train = torch.cat(label_real_train, dim=0).to(device)



In [ ]:
img_real_train.shape


In [ ]:
label_real_train.shape

In [ ]:
img_syn.shape

In [ ]:
# Numpy
import numpy as np

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

# Torchvision
import torchvision
import torchvision.transforms as transforms

# Matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# OS
import os
import argparse

%env CUDA_VISIBLE_DEVICES=1

# Set random seed for reproducibility
SEED = 87
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


def print_model(encoder, decoder):
    print("============== Encoder ==============")
    print(encoder)
    print("============== Decoder ==============")
    print(decoder)
    print("")


def create_model():
    autoencoder = Autoencoder()
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder


def get_torch_vars(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

def imshow(img):
    npimg = img.cpu().numpy()
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),           # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.Conv2d(24, 48, 4, stride=2, padding=1),           # [batch, 48, 4, 4]
            nn.ReLU(),
# 			nn.Conv2d(48, 96, 4, stride=2, padding=1),           # [batch, 96, 2, 2]
#             nn.ReLU(),
        )
        self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1),  # [batch, 48, 4, 4]
#             nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded



In [ ]:
autoencoder = create_model().to(device)

# Load data
transform = transforms.Compose(
    [transforms.ToTensor(), ])

In [ ]:
# autoencoder = torch.load('autoencoder_all.pt').to(device)

In [ ]:

num_classes = 10
batch = 50
num_feat = 3072
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters())
# model = Autoencoder(num_feat).to(device)  
# 训练
for epoch in range(1000):

  total_loss = 0
  for c in range(num_classes):
                # 获取类别c的合成图像和类别中心
                # image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
    batch_img = img_real_train[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    # batch_syn = img_syn[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    # ============ Forward ============
    encoded, outputs = autoencoder(batch_img)
    # print((outputs + batch_img).shape)
    loss = criterion(outputs, batch_img)
    # print(loss.item())
    # ============ Backward ============
    # print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
accs = []
model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)
import copy
accs_all_exps = dict() # record performances of all experiments
for key in model_eval_pool:
    accs_all_exps[key] = []
args.dsa_param = ParamDiffAug()
args.dsa = False if args.dsa_strategy in ['none', 'None'] else True
model_eval= model_eval_pool[0]

In [ ]:
# A - 输入测试图，生成图，label同测试图
with torch.no_grad():
  output = autoencoder(img_real_test.to(device))
data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
# print()
# print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

In [ ]:
# A
autoencoder = create_model().to(device)
num_classes = 10
batch = 50
num_feat = 3072
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters())
# model = Autoencoder(num_feat).to(device)  
# 训练
for epoch in range(3000):

  total_loss = 0
  for c in range(num_classes):
                # 获取类别c的合成图像和类别中心
                # image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
    batch_img = images_all[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    # batch_syn = img_syn[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    # ============ Forward ============
    encoded, outputs = autoencoder(batch_img)
    # print((outputs + batch_img).shape)
    loss = criterion(outputs, batch_img)
    # print(loss.item())
    # ============ Backward ============
    # print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
# A - 输入测试图，生成图，label同测试图
with torch.no_grad():
  output = autoencoder(img_real_test.to(device))
data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
# print()
# print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

In [ ]:
torch.save(autoencoder,'autoencoder_all.pt')

In [ ]:
num_classes = 10
batch = 50
num_feat = 3072
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters())
# model = Autoencoder(num_feat).to(device)  
# 训练
for epoch in range(1000):

  total_loss = 0
  for c in range(num_classes):
                # 获取类别c的合成图像和类别中心
                # image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
    batch_img = img_real_train[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    batch_syn = img_syn[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    # ============ Forward ============
    encoded, outputs = autoencoder(batch_img)
    # print((outputs + batch_img).shape)
    loss = criterion(outputs, batch_syn)
    # print(loss.item())
    # ============ Backward ============
    # print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
# A - 输入测试图，生成图，label同测试图
with torch.no_grad():
  output = autoencoder(img_real_test.to(device))
data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
# print()
# print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

In [ ]:
autoencoder = torch.load('autoencoder_all.pt')

In [ ]:
num_classes = 10
batch = 50
num_feat = 3072
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters())
# model = Autoencoder(num_feat).to(device)  
# 训练
for epoch in range(1000):

  total_loss = 0
  for c in range(num_classes):
                # 获取类别c的合成图像和类别中心
                # image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
    # batch_img = img_real_train[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    batch_syn = img_syn[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    # ============ Forward ============
    encoded, outputs = autoencoder(batch_syn)
    # print((outputs + batch_img).shape)
    loss = criterion(outputs, batch_syn)
    # print(loss.item())
    # ============ Backward ============
    # print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
# 从训练时没见过的原图开始，生成的合成数据性能 训练1W8,test 4500 - 0.3503
# 从训练时见过的原图开始，生成的合成数据性能 训练1W8,test 4500 - 0.3487

# A + FINETUNE + 从训练时没见过的原图开始，生成的合成数据性能 训练1W8,test 4500 -  0.3546
with torch.no_grad():
  output = autoencoder(img_real_test.to(device))
data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
# print()
# print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

In [ ]:
autoencoder = create_model().to(device)
num_classes = 10
batch = 50
num_feat = 3072
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters())
# model = Autoencoder(num_feat).to(device)  
# 训练
for epoch in range(1000):

  total_loss = 0
  for c in range(num_classes):
                # 获取类别c的合成图像和类别中心
                # image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
    batch_img = img_real_train[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    batch_syn = img_syn[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    # ============ Forward ============
    encoded, outputs = autoencoder(batch_img)
    # print((outputs + batch_img).shape)
    loss = criterion(outputs, batch_syn)
    # print(loss.item())
    # ============ Backward ============
    # print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
# 从训练时没见过的原图开始，生成的合成数据性能 训练1W8,test 4500 - 0.3503
# 从训练时见过的原图开始，生成的合成数据性能 训练1W8,test 4500 - 0.3487

# A + FINETUNE + 从训练时没见过的原图开始，生成的合成数据性能 训练1W8,test 4500 -  0.3546
with torch.no_grad():
  output = autoencoder(img_real_test.to(device))
data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
# print()
# print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

In [37]:

# test 合成数据集结果（1500张）
# 直接输入原图： 0.5100
# 样本对训练原图 - 合成图差异： 0.5093


# A+合成图 FINTUNE :    0.4056 - 0.005  |   0.3983 0.4110 0.4075
# A :                  0.3860 - 0.005  |   0.3918 0.3859 0.3802
# A+样本对 FINTUNE :   0.3840 - 0.004  |   0.3885 0.3785 0.3850
# 只用样本对中的原图 :  0.3808 - 0.006  |   0.3780 0.3896 0.3749
# 只用样本对训 ：       0.3719, 0.005 |   0.3642,0.3756,0.3759

accs = [0.3642,0.3756,0.3759]np.mean(accs), np.std(accs)

In [38]:
np.mean(accs), np.std(accs)

(0.37189999999999995, 0.00544609952167603)

In [ ]:
num_classes = 10
batch = 50
num_feat = 3072
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters())
# model = Autoencoder(num_feat).to(device)  
# 训练
for epoch in range(1000):

  total_loss = 0
  for c in range(num_classes):
                # 获取类别c的合成图像和类别中心
                # image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
    batch_img = img_real_train[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    batch_syn = img_syn[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)).to(device) 
    
    # ============ Forward ============
    encoded, outputs = autoencoder(batch_img)
    # print((outputs + batch_img).shape)
    loss = criterion(outputs, batch_syn - batch_img)
    # print(loss.item())
    # ============ Backward ============
    # print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
accs = []
model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)
import copy
accs_all_exps = dict() # record performances of all experiments
for key in model_eval_pool:
    accs_all_exps[key] = []
args.dsa_param = ParamDiffAug()
args.dsa = False if args.dsa_strategy in ['none', 'None'] else True
model_eval= model_eval_pool[0]

In [ ]:
# 从训练时没见过的原图开始，生成的合成数据性能 训练1W8,test 4500 - 0.3503
# 从训练时见过的原图开始，生成的合成数据性能 训练1W8,test 4500 - 0.3487

# A + FINETUNE + 从训练时没见过的原图开始，生成的合成数据性能 训练1W8,test 4500 -  0.3546
with torch.no_grad():
  output = autoencoder(img_real_test.to(device))
data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1] + img_real_test), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
# print()
# print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

In [ ]:
torch.save('autoencoder_all.pt')

In [ ]:

data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(img_real_test), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
# print()
# print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

In [ ]:
# test 原图： 0.5100
# 样本对训练原图 - 合成图差异： 0.5093
# A : 0.3791
# A+样本对 FINTUNE : 
# A+合成图 FINTUNE : 
# 样本对 ： 

In [ ]:
image_syn_vis = copy.deepcopy(output[1].detach().cpu()+ img_real_test.cpu())
from torchvision.utils import save_image
image_syn_vis[image_syn_vis<0] = 0.0
image_syn_vis[image_syn_vis>1] = 1.0
save_image(image_syn_vis, "/home/wangkai/ztl_project/difussion-dd/DatasetCondensation-master/111.png", nrow=50) # Trying normalize = True/False may get better visual effects.

In [ ]:
import torch
import copy

In [ ]:
img_syn_ = torch.load(f'/home/ssd7T/ztl_ftd/img_syn_57.pt')
# /home/ssd7T/ztl_ftd/img_syn_57.pt

In [ ]:
buffer = torch.load(f'/home/ssd7T/ztl_dm/mtt/CIFAR10/ConvNet/replay_buffer_0.pt')
# /home/ssd7T/ztl_ftd/img_syn_57.pt

In [ ]:
print(len(buffer))

In [ ]:
image_syn_vis = copy.deepcopy(img_syn_.detach().cpu())
from torchvision.utils import save_image
image_syn_vis[image_syn_vis<0] = 0.0
image_syn_vis[image_syn_vis>1] = 1.0
save_image(image_syn_vis, "/home/wangkai/ztl_project/difussion-dd/DatasetCondensation-master/333.png", nrow=50) # Trying normalize = True/False may get better visual effects.